## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
prefix_path = '../data/mimiciv/2.0/'

## Load Data

In [3]:
paths = sorted(os.listdir(prefix_path))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('procedureevents.csv')
paths.remove('datetimeevents.csv')
paths.remove('d_hcpcs.csv')
paths.remove('transfers.csv')
paths.remove('procedures_icd.csv')
paths.remove('emar_detail.csv')
paths.remove('emar.csv')
paths.remove('services.csv')
paths.remove('microbiologyevents.csv')
paths.remove('d_icd_procedures.csv')
paths.remove('pharmacy.csv')
paths.remove('omr.csv')
paths.remove('poe.csv')
paths.remove('poe_detail.csv')
paths.remove('hcpcsevents.csv')
paths.remove('ingredientevents.csv')
paths.remove('drgcodes.csv')
paths

['admissions.csv',
 'chartevent1.csv',
 'chartevent10.csv',
 'chartevent11.csv',
 'chartevent2.csv',
 'chartevent3.csv',
 'chartevent4.csv',
 'chartevent5.csv',
 'chartevent6.csv',
 'chartevent7.csv',
 'chartevent8.csv',
 'chartevent9.csv',
 'd_icd_diagnoses.csv',
 'd_items.csv',
 'd_labitems.csv',
 'diagnoses_icd.csv',
 'icustays.csv',
 'inputevents.csv',
 'labevents.csv',
 'outputevents.csv',
 'patients.csv',
 'prescriptions.csv']

In [4]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv(prefix_path + path))

 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉               | 20/22 [01:15<00:05,  2.98s/it]/tmp/ipykernel_139259/136570088.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(prefix_path + path))
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [01:17<00:00,  3.53s/it]


## Clean Data to remain SEPSIS patient and sort Data

In [5]:
i = paths.index('d_icd_diagnoses.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES

,Unnamed: 0,icd_code,icd_version,long_title
0,0,0010,9,Cholera due to vibrio cholerae
1,1,0011,9,Cholera due to vibrio cholerae el tor
2,2,0019,9,"Cholera, unspecified"
3,3,0020,9,Typhoid fever
4,4,0021,9,Paratyphoid fever A
...,...,...,...,...
109770,109770,Z992,10,Dependence on renal dialysis
109771,109771,Z993,10,Dependence on wheelchair
109772,109772,Z998,10,Dependence on other enabling machines and devices
109773,109773,Z9981,10,Dependence on supplemental oxygen


In [6]:
i = paths.index('diagnoses_icd.csv')
datas[i].sort_values(by=['subject_id', 'seq_num'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
DIAGNOSES_ICD

,Unnamed: 0,subject_id,hadm_id,seq_num,icd_code,icd_version
0,135,10000826,21086876,2,99591,9
1,537,10001401,27012892,2,A4181,10
2,553,10001401,27060146,2,A419,10
3,538,10001401,27012892,3,R6520,10
4,542,10001401,27012892,7,T8144XA,10
...,...,...,...,...,...,...
21093,5002755,19992202,20329411,16,99592,9
21094,5005349,19997367,20617667,4,99592,9
21095,5005416,19997367,21508795,18,99592,9
21096,5005780,19997886,20793010,4,A419,10


In [7]:
for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'time' in column or 'date' in column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [05:15<00:00, 14.36s/it]


# Look Data characteristics

In [8]:
i = paths.index('icustays.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS.head()

,Unnamed: 0,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,13,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222
1,14,10002428,28662225,33987268,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-12 16:24:18,2156-04-17 15:57:08,4.981134
2,15,10002428,28662225,38875437,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2156-04-19 18:11:19,2156-04-26 18:58:41,7.032894
3,24,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213
4,27,10003400,23559586,34577403,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-08-10 19:54:51,2137-08-13 17:54:54,2.916701


In [9]:
i = paths.index('patients.csv')
PATIENTS = datas[i].copy()
PATIENTS.head()

,Unnamed: 0,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,25,10000826,F,32,2146,2008 - 2010,NaN
1,46,10001401,F,89,2131,2014 - 2016,NaN
2,67,10002013,F,53,2156,2008 - 2010,NaN
3,79,10002428,F,80,2155,2011 - 2013,NaN
4,111,10003019,M,69,2171,2008 - 2010,NaN


In [10]:
i = paths.index('admissions.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS.head()

,Unnamed: 0,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,18,10000826,21086876,2146-12-18 17:39:00,2146-12-24 19:55:00,NaT,DIRECT EMER.,CLINIC REFERRAL,HOME,Other,ENGLISH,SINGLE,WHITE,NaT,NaT,0
1,55,10001401,27012892,2133-07-09 22:22:00,2133-07-13 18:43:00,NaT,OBSERVATION ADMIT,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaT,NaT,0
2,56,10001401,27060146,2131-10-01 01:33:00,2131-10-05 15:45:00,NaT,OBSERVATION ADMIT,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2131-09-30 16:47:00,2131-10-01 02:24:00,0
3,96,10002013,24848509,2162-07-08 00:08:00,2162-07-09 18:06:00,NaT,EW EMER.,EMERGENCY ROOM,HOME,Medicare,ENGLISH,SINGLE,WHITE,2162-07-07 19:40:00,2162-07-08 01:45:00,0
4,97,10002013,25442395,2166-04-06 18:47:00,2166-04-19 16:38:00,NaT,OBSERVATION ADMIT,CLINIC REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2166-04-06 16:01:00,2166-04-06 21:21:00,0


In [11]:
i = paths.index('inputevents.csv')
INPUTEVENTS_CV = datas[i].copy()
INPUTEVENTS_CV.head()

,Unnamed: 0,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,amount,amountuom,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,705,10002428,23473524,35479615,2156-05-11 16:30:00,2156-05-11 21:51:00,2156-05-11 17:43:00,221906,1.554480,mg,...,Main order parameter,Continuous Med,48.4,250.0,ml,0,0,ChangeDose/Rate,8.0,0.1000
1,706,10002428,23473524,35479615,2156-05-11 16:30:00,2156-05-11 21:51:00,2156-05-11 17:43:00,225158,48.577483,ml,...,Mixed solution,Continuous Med,48.4,250.0,ml,0,0,ChangeDose/Rate,250.0,9.0744
2,707,10002428,23473524,35479615,2156-05-11 17:15:00,2156-05-11 17:16:00,2156-05-11 17:43:00,220949,200.000000,ml,...,Mixed solution,Drug Push,48.4,200.0,ml,0,0,FinishedRunning,200.0,0.0000
3,708,10002428,23473524,35479615,2156-05-11 17:15:00,2156-05-11 17:16:00,2156-05-11 17:43:00,225859,1.000000,dose,...,Main order parameter,Drug Push,48.4,200.0,ml,0,0,FinishedRunning,1.0,1.0000
4,709,10002428,23473524,35479615,2156-05-11 18:59:00,2156-05-11 19:00:00,2156-05-11 18:59:00,220949,50.000000,ml,...,Mixed solution,Drug Push,48.4,50.0,ml,0,0,FinishedRunning,50.0,0.0000


In [12]:
i = paths.index('outputevents.csv')
OUTPUTEVENTS = datas[i].copy()
OUTPUTEVENTS.head()

,Unnamed: 0,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valueuom
0,483,10002428,23473524,35479615,2156-05-11 17:44:00,2156-05-11 17:45:00,226559,350.0,ml
1,484,10002428,23473524,35479615,2156-05-11 20:00:00,2156-05-11 19:49:00,226559,45.0,ml
2,485,10002428,23473524,35479615,2156-05-11 21:00:00,2156-05-11 22:14:00,226559,55.0,ml
3,486,10002428,23473524,35479615,2156-05-11 22:00:00,2156-05-11 22:14:00,226559,75.0,ml
4,487,10002428,23473524,35479615,2156-05-11 23:00:00,2156-05-12 00:16:00,226559,100.0,ml


In [13]:
i = paths.index('d_items.csv')
D_ITEMS = datas[i].copy()
D_ITEMS.head()

,Unnamed: 0,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


In [14]:
i = paths.index('d_labitems.csv')
D_LABITEMS = datas[i].copy()
D_LABITEMS.head()

,Unnamed: 0,itemid,label,fluid,category
0,0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,1,50802,Base Excess,Blood,Blood Gas
2,2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,3,50804,Calculated Total CO2,Blood,Blood Gas
4,4,50805,Carboxyhemoglobin,Blood,Blood Gas


In [15]:
i = paths.index('labevents.csv')
LABEVENTS = datas[i].copy()
LABEVENTS.head()

,Unnamed: 0,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,3045,3051,10000826,21086876.0,45348866,51116,2146-12-18 18:33:00,2146-12-18 20:38:00,54,54.0,%,0.0,0.0,abnormal,STAT,NaN
1,3046,3052,10000826,21086876.0,45348866,51117,2146-12-18 18:33:00,2146-12-18 20:38:00,16,16.0,%,0.0,0.0,abnormal,STAT,NaN
2,3047,3053,10000826,21086876.0,45348866,51118,2146-12-18 18:33:00,2146-12-18 20:38:00,1,1.0,%,0.0,0.0,abnormal,STAT,NaN
3,3048,3054,10000826,21086876.0,45348866,51120,2146-12-18 18:33:00,2146-12-18 20:38:00,0,0.0,%,0.0,0.0,NaN,STAT,NaN
4,3049,3055,10000826,21086876.0,45348866,51125,2146-12-18 18:33:00,2146-12-18 20:38:00,29,29.0,%,0.0,0.0,abnormal,STAT,NaN


In [16]:
i = paths.index('prescriptions.csv')
PRESCRIPTIONS = datas[i].copy()
PRESCRIPTIONS.head()

,Unnamed: 0,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,286,10000826,21086876,2200640,10000826-172,172.0,2146-12-18 21:00:00,2146-12-19 20:00:00,MAIN,Albumin 25% (12.5g / 50mL),...,006329,6.746706e+10,25%-50mL,NaN,50,g,4,BAG,1.0,IV
1,287,10000826,21086876,5536597,10000826-222,222.0,2146-12-20 16:00:00,2146-12-21 15:00:00,MAIN,Albumin 25% (12.5g / 50mL),...,006329,6.746706e+10,25%-50mL,NaN,25,g,2,BAG,1.0,IV
2,288,10000826,21086876,15358102,10000826-178,178.0,2146-12-18 21:00:00,2146-12-19 21:00:00,MAIN,OxycoDONE (Immediate Release),...,004225,4.060553e+08,5mg Tablet,NaN,5-10,mg,1-2,TAB,NaN,PO/NG
3,289,10000826,21086876,19616931,10000826-167,167.0,2146-12-18 10:00:00,2146-12-25 00:00:00,MAIN,FoLIC Acid,...,002366,1.820508e+08,1 mg Tab,NaN,1,mg,1,TAB,1.0,PO/NG
4,290,10000826,21086876,19698853,NaN,NaN,2146-12-24 02:00:00,2146-12-25 00:00:00,MAIN,OxycoDONE (Immediate Release),...,004225,4.060553e+08,5mg Tablet,NaN,10,mg,2,TAB,NaN,PO


In [18]:
i = paths.index('chartevent1.csv')
CHARTEVENTS1 = datas[i].copy()
i = paths.index('chartevent2.csv')
CHARTEVENTS2 = datas[i].copy()
i = paths.index('chartevent3.csv')
CHARTEVENTS3 = datas[i].copy()
i = paths.index('chartevent4.csv')
CHARTEVENTS4 = datas[i].copy()
i = paths.index('chartevent5.csv')
CHARTEVENTS5 = datas[i].copy()
i = paths.index('chartevent6.csv')
CHARTEVENTS6 = datas[i].copy()
i = paths.index('chartevent7.csv')
CHARTEVENTS7 = datas[i].copy()
i = paths.index('chartevent8.csv')
CHARTEVENTS8 = datas[i].copy()
i = paths.index('chartevent9.csv')
CHARTEVENTS9 = datas[i].copy()
i = paths.index('chartevent10.csv')
CHARTEVENTS10 = datas[i].copy()
i = paths.index('chartevent11.csv')
CHARTEVENTS11 = datas[i].copy()

# Features

In [19]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Add Albumin to dataset

In [18]:
# 50862, 772, 1521, 227456

In [21]:
D_ITEMS.drop(D_ITEMS[D_ITEMS.isnull()['label']].index, inplace=True)
D_ITEMS[['albumin' in str(s).lower() for s in D_ITEMS['label']]]

,Unnamed: 0,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
141,141,220574,ZAlbumin,ZAlbumin,chartevents,Labs,None,Numeric,NaN,NaN
162,162,220861,Albumin (Human) 20%,Albumin 20%,inputevents,Fluids - Other (Not In Use),mL,Solution,NaN,NaN
163,163,220862,Albumin 25%,Albumin 25%,inputevents,Blood Products/Colloids,mL,Solution,NaN,NaN
164,164,220863,Albumin (Human) 4%,Albumin 4%,inputevents,Fluids - Other (Not In Use),mL,Solution,NaN,NaN
165,165,220864,Albumin 5%,Albumin 5%,inputevents,Blood Products/Colloids,mL,Solution,NaN,NaN
2025,2025,226981,Albumin_ApacheIV,Albumin_ApacheIV,chartevents,Scores - APACHE IV (2),None,Numeric,NaN,NaN
2026,2026,226982,AlbuminScore_ApacheIV,AlbuminScore_ApacheIV,chartevents,Scores - APACHE IV (2),None,Numeric,NaN,NaN
2217,2217,227456,Albumin,Albumin,chartevents,Labs,None,Numeric with tag,NaN,NaN


In [22]:
D_LABITEMS[['albumin' in str(s).lower() for s in D_LABITEMS['label']]]

,Unnamed: 0,itemid,label,fluid,category
33,33,50835,"Albumin, Ascites",Ascites,Chemistry
60,60,50862,Albumin,Blood,Chemistry
208,208,51019,"Albumin, Joint Fluid",Joint Fluid,Chemistry
214,214,51025,"Albumin, Body Fluid",Other Body Fluid,Chemistry
235,235,51046,"Albumin, Pleural",Pleural,Chemistry
258,258,51069,"Albumin, Urine",Urine,Chemistry
259,259,51070,"Albumin/Creatinine, Urine",Urine,Chemistry
895,895,51775,(Albumin),Cerebrospinal Fluid,Chemistry
896,896,51776,<Albumin>,Cerebrospinal Fluid,Chemistry
897,897,51777,"Albumin, CSF",Cerebrospinal Fluid,Chemistry


## Add Arterial_pH to dataset

In [21]:
# 780, 1126, 223830, 50820

In [25]:
D_ITEMS[['art' in str(s).lower() and 'ph' in str(s).lower() for s in D_ITEMS['label']]]

,Unnamed: 0,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
382,382,223830,PH (Arterial),PH (Arterial),chartevents,Labs,None,Numeric,NaN,NaN
1072,1072,225076,Emotional / physical / sexual harm by partner ...,Emotional / physical / sexual harm,chartevents,Adm History/FHPA,NaN,Checkbox,NaN,NaN
2734,2734,228194,Peripheral Pulses (Tandem Heart),Peripheral Pulses (Tandem Heart),chartevents,Tandem Heart,NaN,Text,NaN,NaN


In [30]:
D_LABITEMS[['ph' in str(s).lower() for s in D_LABITEMS['label']]]

,Unnamed: 0,itemid,label,fluid,category
18,18,50820,pH,Blood,Blood Gas
29,29,50831,pH,Other Body Fluid,Blood Gas
54,54,50856,Acetaminophen,Blood,Chemistry
56,56,50858,Acid Phosphatase,Blood,Chemistry
57,57,50859,"Acid Phosphatase, Non-Prostatic",Blood,Chemistry
...,...,...,...,...,...
1605,1605,53135,"Acid Phosphatase, Prostatic Fraction",Blood,Chemistry
1606,1606,53136,Adrenocorticotrophic Hormone,Blood,Chemistry
1611,1611,53141,Alpha-1,Blood,Chemistry
1612,1612,53142,Alpha-1-antitrypsin,Blood,Chemistry


## Add Calcium to dataset 

In [26]:
# 225625, 50893, 786, 1522

In [27]:
D_ITEMS[['calcium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
679,1525,1525,706,786,Calcium (8.4-10.2),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
708,1554,1554,735,816,Ionized Calcium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
959,1231,1231,1360,1350,ionized calcium,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1111,2790,2790,2974,1522,Calcium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1249,2928,2928,3112,1663,Calcium gluconate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1571,3065,3065,3434,2185,CALCIUM GLUCONATE,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2218,1687,1687,868,3219,calcium gluconate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2311,1780,1780,961,3347,Calcium Gluconate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2594,2684,2684,2868,3746,Calcium (8.8-10.8),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2606,2696,2696,2880,3766,Ion Calcium,NaN,carevue,chartevents,ABG's,NaN,NaN,NaN


In [28]:
D_LABITEMS[['calcium' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
8,135,135,9,50808,Free Calcium,Blood,Blood Gas,1994-3
93,220,220,94,50893,"Calcium, Total",Blood,Chemistry,2000-8
228,355,355,229,51029,"Calcium, Body Fluid",Other Body Fluid,Chemistry,15155-5
265,392,392,266,51066,24 hr Calcium,Urine,Chemistry,6874-2
276,403,403,277,51077,"Calcium, Urine",Urine,Chemistry,2004-0
667,122,122,668,51468,Calcium Carbonate Crystals,Urine,Hematology,5773-7
668,123,123,669,51469,Calcium Oxalate Crystals,Urine,Hematology,5774-5
669,124,124,670,51470,Calcium Phosphate Crystals,Urine,Hematology,5775-2


## Add Glucose to dataset  

In [ ]:
# 220621, 225664, 50809, 811, 807, 50931, 226537, 1529

In [39]:
D_ITEMS[['glucose' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
699,1545,1545,726,807,Fingerstick Glucose,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
703,1549,1549,730,811,Glucose (70-105),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
926,1198,1198,1327,1310,FINGERSTICK GLUCOSE.,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1051,277,277,14,1455,fingerstick glucose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1118,2797,2797,2981,1529,Glucose,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1342,1971,1971,3205,1812,abg: glucose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1625,3119,3119,3488,2338,finger stick glucose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2390,1859,1859,1040,3447,Glucose Monitor #,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2592,2682,2682,2866,3744,Blood Glucose,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2593,2683,2683,2867,3745,BloodGlucose,NaN,carevue,chartevents,Quick Admit,NaN,NaN,NaN


In [40]:
D_LABITEMS[['glucose' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
9,136,136,10,50809,Glucose,Blood,Blood Gas,2339-0
42,169,169,43,50842,"Glucose, Ascites",Ascites,Chemistry,2347-3
131,258,258,132,50931,Glucose,Blood,Chemistry,2345-7
213,340,340,214,51014,"Glucose, CSF",Cerebrospinal Fluid (CSF),Chemistry,2342-4
221,348,348,222,51022,"Glucose, Joint Fluid",Joint Fluid,Chemistry,2348-1
233,360,360,234,51034,"Glucose, Body Fluid",Other Body Fluid,Chemistry,2344-0
252,379,379,253,51053,"Glucose, Pleural",Pleural,Chemistry,2346-5
283,410,410,284,51084,"Glucose, Urine",Urine,Chemistry,2350-7
677,677,677,678,51478,Glucose,Urine,Hematology,5792-7
728,728,728,729,51529,Estimated Actual Glucose,BLOOD,CHEMISTRY,NaN


## Add Hemoglobin to dataset  

In [29]:
# 220228, 51222, 50811, 814

In [30]:
D_ITEMS[['hemoglobin' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
706,1552,1552,733,814,Hemoglobin,NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
4669,982,982,2336,7965,methhemoglobin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9526,11528,11528,12742,220228,Hemoglobin,Hemoglobin,metavision,chartevents,Labs,g/dl,Numeric,NaN


In [31]:
D_LABITEMS[['hemoglobin' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
5,132,132,6,50805,Carboxyhemoglobin,Blood,Blood Gas,20563-3
11,138,138,12,50811,Hemoglobin,Blood,Blood Gas,718-7
14,141,141,15,50814,Methemoglobin,Blood,Blood Gas,2614-6
52,179,179,53,50852,% Hemoglobin A1c,Blood,Chemistry,4548-4
55,182,182,56,50855,Absolute Hemoglobin,Blood,Chemistry,718-7
411,538,538,412,51212,Fetal Hemoglobin,Blood,Hematology,4576-5
421,548,548,422,51222,Hemoglobin,Blood,Hematology,718-7
422,549,549,423,51223,Hemoglobin A2,Blood,Hematology,4552-6
423,550,550,424,51224,Hemoglobin C,Blood,Hematology,4561-7
424,551,551,425,51225,Hemoglobin F,Blood,Hematology,9749-3


## Add  Magnesium to dataset  

In [32]:
# 220635, 50960, 1532, 821

In [33]:
D_ITEMS[['magnesium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
713,1559,1559,740,821,Magnesium (1.6-2.6),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1121,2800,2800,2984,1532,Magnesium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1450,2079,2079,3313,1970,magnesium 2gm/hr,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3769,3616,3616,2539,6133,Magnesium Sulfate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5058,5783,5783,5353,30027,Magnesium Sulfate,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5492,6076,6076,5793,40645,magnesium,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
6643,6769,6769,8039,43018,magnesium 1gm/hr,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7126,6246,6246,9086,44088,Magnesium,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
9554,11488,11488,12702,220635,Magnesium,Magnesium,metavision,chartevents,Labs,None,Numeric,NaN
9690,9555,9555,12986,222011,Magnesium Sulfate,Magnesium Sulfate,metavision,inputevents_mv,Medications,grams,Solution,NaN


In [34]:
D_LABITEMS[['magnesium' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
160,287,287,161,50960,Magnesium,Blood,Chemistry,2601-3
236,363,363,237,51037,"Magnesium, Body Fluid",Other Body Fluid,Chemistry,29365-4
287,414,414,288,51088,"Magnesium, Urine",Urine,Chemistry,2598-1


## Add  PTT-Partial Thromboplastin Time to dataset  

In [35]:
# 227466, 825, 1533, 51275

In [36]:
D_ITEMS[['ptt' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
717,1563,1563,744,825,PTT(22-35),NaN,carevue,chartevents,Coags,NaN,NaN,NaN
1122,2801,2801,2985,1533,PTT,NaN,carevue,chartevents,Coags,NaN,NaN,NaN
2625,2715,2715,2899,3796,Ptt,NaN,carevue,chartevents,Heme/Coag,NaN,NaN,NaN
9541,11475,11475,12689,220562,ZPTT,ZPTT,metavision,chartevents,Labs,None,Numeric,NaN
11619,10305,10305,14827,227466,PTT,PTT,metavision,chartevents,Labs,None,Numeric with tag,NaN


In [37]:
D_LABITEMS[['ptt' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
474,601,601,475,51275,PTT,Blood,Hematology,3173-2


## Add  Potassium to dataset 

In [38]:
# 829, 50971, 1535, 227464, 50822

In [39]:
D_ITEMS[['potassium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
721,1567,1567,748,829,Potassium (3.5-5.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1124,2803,2803,2987,1535,Potassium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1349,1978,1978,3212,1821,potassium phosphate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1860,3354,3354,3723,2691,URINE POTASSIUM,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1969,3463,3463,3832,2856,Potassium phosphate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2489,3840,3840,2763,3578,Potassium Chloride,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2580,3931,3931,2854,3725,ABG POTASSIUM,NaN,carevue,chartevents,ABG'S,NaN,NaN,NaN
2623,2713,2713,2897,3792,Potassium (3.5-5.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2668,870,870,2224,4194,ABG Potassium,NaN,carevue,chartevents,ABG'S,NaN,NaN,NaN
2771,1311,1311,1440,4381,Potassium Phosphate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [40]:
D_LABITEMS[['potassium' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
22,149,149,23,50822,"Potassium, Whole Blood",Blood,Blood Gas,6298-4
33,160,160,34,50833,Potassium,Other Body Fluid,Blood Gas,2821-7
47,174,174,48,50847,"Potassium, Ascites",Ascites,Chemistry,49789-1
171,298,298,172,50971,Potassium,Blood,Chemistry,2823-3
240,367,367,241,51041,"Potassium, Body Fluid",Other Body Fluid,Chemistry,2821-7
256,383,383,257,51057,"Potassium, Pleural",Pleural,Chemistry,NaN
263,390,390,264,51064,"Potassium, Stool",Stool,Chemistry,15202-5
296,423,423,297,51097,"Potassium, Urine",Urine,Chemistry,2828-2


## Add  SGPT-Serum Glutamic-Pyruvic Transaminase to dataset 

In [42]:
# 3802

In [43]:
D_ITEMS[['sgpt' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2629,2719,2719,2903,3802,SGPT,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [44]:
# only one id
sgpt_id = D_ITEMS[['sgpt' in str(s).lower() for s in D_ITEMS['LABEL']]]['ITEMID'].iloc[0]

## Add  Arterial blood gas to dataset 

In [51]:
D_ITEMS[['arterial' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
30,320,320,57,51,Arterial BP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
31,321,321,58,52,Arterial BP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
32,322,322,59,53,Arterial Pressure,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
669,1515,1515,696,776,Arterial Base Excess,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
670,1516,1516,697,777,Arterial CO2(Calc),NaN,carevue,chartevents,ABG,NaN,NaN,NaN
671,1517,1517,698,778,Arterial PaCO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
672,1518,1518,699,779,Arterial PaO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
673,1519,1519,700,780,Arterial pH,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
1045,271,271,8,1449,Arterial BP(Rad),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1757,3251,3251,3620,2529,arterial pulse flap,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [52]:
D_LABITEMS[['arterial' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
1,128,128,2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9


## Add  BUN-Blood Urea Nitrogen to dataset 

In [45]:
# 51006, 781, 1162, 225624

In [46]:
D_ITEMS[['bun' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
674,1520,1520,701,781,BUN (6-20),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
876,125,125,1277,1162,BUN,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2588,3939,3939,2862,3737,BUN (6-20),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
3527,2522,2522,1976,5876,bun,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4773,4025,4025,4025,8220,Effluent BUN,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10797,10505,10505,14081,225624,BUN,BUN,metavision,chartevents,Labs,None,Numeric,NaN
11438,9668,9668,14677,227000,BUN_ApacheIV,BUN_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN
11439,9669,9669,14678,227001,BunScore_ApacheIV,BunScore_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN


In [47]:
D_LABITEMS[D_LABITEMS['ITEMID'] == 51006]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
205,332,332,206,51006,Urea Nitrogen,Blood,Chemistry,3094-0


## Add  HCO3 to dataset 

In [48]:
# 50882, 227443, 50803

In [49]:
D_ITEMS[['hco3' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
704,1550,1550,731,812,HCO3,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
1710,3204,3204,3573,2468,NAHCO3- MEQ/HR,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4777,4029,4029,4029,8235,naHCO3,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5903,5967,5967,6338,41824,D5W 150meq NAHCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
5956,8554,8554,10799,41983,NAHCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
6170,5403,5403,7532,42337,NAHCO3- CC/HR,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
6590,6722,6722,7971,42939,1/4ns w 100meqnahco3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7017,6166,6166,8952,43904,D51/4NS+100MeqNaHCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7179,6291,6291,9152,44166,BICARBONATE-HCO3,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7398,6937,6937,8313,44539,D5W W/ NAHCO3 150MEQ,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN


In [50]:
D_LABITEMS.query("ITEMID == 50882 | ITEMID == 50803")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
3,130,130,4,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,1959-6
82,209,209,83,50882,Bicarbonate,Blood,Chemistry,1963-8


## Add  INR-International Normalized Ratio to dataset 

In [55]:
D_ITEMS[['inr' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
707,1553,1553,734,815,INR (2-4 ref. range),NaN,carevue,chartevents,Coags,NaN,NaN,NaN
1119,2798,2798,2982,1530,INR,NaN,carevue,chartevents,Coags,NaN,NaN,NaN
6310,5504,5504,7669,42527,INR BLOOD PRODUCTS,NaN,carevue,inputevents_cv,Free Form Intake,NaN,NaN,NaN
7355,6897,6897,8258,44481,ER URINR OUTPUT,NaN,carevue,outputevents,NaN,NaN,NaN,NaN
9540,11474,11474,12688,220561,ZINR,ZINR,metavision,chartevents,Labs,None,Numeric,NaN
11620,10306,10306,14828,227467,INR,INR,metavision,chartevents,Labs,None,Numeric with tag,NaN


## Add  Arterial Lactate to dataset 

In [51]:
# 50813

In [52]:
D_LABITEMS.query("ITEMID == 50813")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
13,140,140,14,50813,Lactate,Blood,Blood Gas,32693-4


## Add  CO2 to dataset 

In [53]:
# 787

In [54]:
D_ITEMS.query("ITEMID == 787")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
680,1526,1526,707,787,Carbon Dioxide,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [55]:
D_ITEMS[[' co2' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
670,1516,1516,697,777,Arterial CO2(Calc),NaN,carevue,chartevents,ABG,NaN,NaN,NaN
744,768,768,1145,857,Venous CO2(Calc),NaN,carevue,chartevents,VBG'S,NaN,NaN,NaN
1914,3408,3408,3777,2772,SJ CO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2610,2700,2700,2884,3773,Mix Venous CO2(calc),NaN,carevue,chartevents,Mixed VBGs,NaN,NaN,NaN
2634,2724,2724,2908,3810,Total CO2,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2644,2734,2734,2918,3830,Venous CO2,NaN,carevue,chartevents,Venous ABG,NaN,NaN,NaN
9527,11529,11529,12743,220235,Arterial CO2 Pressure,PCO2 (Arterial),metavision,chartevents,Labs,mmHg,Numeric,NaN
11136,11644,11644,14344,226062,Venous CO2 Pressure,PCO2 (Venous),metavision,chartevents,Labs,mmHg,Numeric,NaN


In [58]:
D_LABITEMS[[' co2' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
4,131,131,5,50804,Calculated Total CO2,Blood,Blood Gas,34728-6


## Add  Creatinine to dataset  

In [57]:
# 791, 1525, 220615, 50912

In [58]:
D_ITEMS[['creatinine' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
683,1529,1529,710,791,Creatinine (0-1.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1114,2793,2793,2977,1525,Creatinine,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
2597,2687,2687,2871,3750,Creatinine (0-0.7),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
3465,2460,2460,1914,5811,urine creatinine,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9550,11484,11484,12698,220615,Creatinine,Creatinine,metavision,chartevents,Labs,None,Numeric,NaN
11356,12476,12476,14512,226751,CreatinineApacheIIScore,CreatinineApacheIIScore,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN
11357,12477,12477,14513,226752,CreatinineApacheIIValue,CreatinineApacheIIValue,metavision,chartevents,Scores - APACHE II,None,Numeric,NaN
11443,9673,9673,14682,227005,Creatinine_ApacheIV,Creatinine_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN


In [59]:
D_LABITEMS[['creatinine' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
41,168,168,42,50841,"Creatinine, Ascites",Ascites,Chemistry,12191-3
112,239,239,113,50912,Creatinine,Blood,Chemistry,2160-0
220,347,347,221,51021,"Creatinine, Joint Fluid",Joint Fluid,Chemistry,14401-4
231,358,358,232,51032,"Creatinine, Body Fluid",Other Body Fluid,Chemistry,12190-5
251,378,378,252,51052,"Creatinine, Pleural",Pleural,Chemistry,14399-0
266,393,393,267,51067,24 hr Creatinine,Urine,Chemistry,2162-6
269,396,396,270,51070,"Albumin/Creatinine, Urine",Urine,Chemistry,14958-3
272,399,399,273,51073,"Amylase/Creatinine Ratio, Urine",Urine,Chemistry,34235-2
279,406,406,280,51080,Creatinine Clearance,Urine,Chemistry,33558-8
280,407,407,281,51081,"Creatinine, Serum",Urine,Chemistry,NaN


## Add  Ionised Calcium to dataset 

In [60]:
# 50808, 816, 225667

In [61]:
D_ITEMS[['ionized calcium' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
708,1554,1554,735,816,Ionized Calcium,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
959,1231,1231,1360,1350,ionized calcium,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4753,4005,4005,4005,8177,Ionized calcium,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4811,4063,4063,4063,8325,IONIZED CALCIUM,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10828,10536,10536,14112,225667,Ionized Calcium,Ionized Calcium,metavision,chartevents,Labs,None,Numeric,NaN


In [62]:
D_LABITEMS.query("ITEMID == 50808")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
8,135,135,9,50808,Free Calcium,Blood,Blood Gas,1994-3


## Add  PT-Prothrombin Time to dataset 

In [63]:
# 1286, 824, 227465, 51274

In [64]:
D_ITEMS.query("ITEMID == 1286 | ITEMID == 824 | ITEMID == 227465")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
716,1562,1562,743,824,PT(11-13.5),NaN,carevue,chartevents,Coags,NaN,NaN,NaN
925,1197,1197,1326,1286,PT,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
11618,10304,10304,14826,227465,Prothrombin time,PT,metavision,chartevents,Labs,None,Numeric with tag,NaN


In [65]:
D_LABITEMS.query("ITEMID == 51274")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
473,600,600,474,51274,PT,Blood,Hematology,5902-2


## Add Platelets Count to dataset 

In [66]:
# 51265, 828, 227457

In [67]:
D_ITEMS[['platelet' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
720,1566,1566,747,828,Platelets,NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
2621,2711,2711,2895,3789,Platelet (150-440),NaN,carevue,chartevents,Heme/Coag,NaN,NaN,NaN
3880,4082,4082,4082,6256,Platelet Count,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5039,5770,5770,5340,30006,Platelets,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5127,5830,5830,5405,30105,OR Platelets,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
10508,8679,8679,13768,225170,Platelets,Platelets,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,NaN
10836,10544,10544,14120,225678,ZPlatelet Count,ZPlatelet Count,metavision,chartevents,Labs,None,Numeric,NaN
11219,12262,12262,14454,226369,OR Platelet Intake,OR Platelet Intake,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,NaN
11503,8743,8743,14576,227071,PACU Platelet Intake,PACU Platelet Intake,metavision,inputevents_mv,Blood Products/Colloids,mL,Solution,NaN
11610,10088,10088,14771,227457,Platelet Count,Platelet Count,metavision,chartevents,Labs,None,Numeric with tag,NaN


In [68]:
D_LABITEMS[['platelet' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
439,566,566,440,51240,Large Platelets,Blood,Hematology,34167-7
463,590,590,464,51264,Platelet Clumps,Blood,Hematology,40741-1
464,591,591,465,51265,Platelet Count,Blood,Hematology,777-3
465,592,592,466,51266,Platelet Smear,Blood,Hematology,778-1


## Add SGOT-Serum Glutamic-Oxaloacetic Transaminse to dataset  

In [69]:
# 3801

In [70]:
D_ITEMS[['sgot' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2628,2718,2718,2902,3801,SGOT,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [71]:
# only one id
sgot_id = D_ITEMS[['sgot' in str(s).lower() for s in D_ITEMS['LABEL']]]['ITEMID'].iloc[0]

## Add Total bilirubun to dataset  

In [73]:
# 848, 225690, 1538, 50885

In [74]:
D_ITEMS.query("ITEMID == 848 | ITEMID == 1538 | ITEMID == 803")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
695,1541,1541,722,803,Direct Bili (0-0.3),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
735,759,759,1136,848,Total Bili (0-1.5),NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN
1127,2806,2806,2990,1538,Total Bili,NaN,carevue,chartevents,Chemistry,NaN,NaN,NaN


In [75]:
D_ITEMS[['bilirubin' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
3044,2259,2259,1713,4948,Bilirubin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10819,10527,10527,14103,225651,Direct Bilirubin,Direct Bilirubin,metavision,chartevents,Labs,None,Numeric,NaN
10845,10553,10553,14129,225690,Total Bilirubin,Total Bilirubin,metavision,chartevents,Labs,None,Numeric,NaN
11436,9666,9666,14675,226998,Bilirubin_ApacheIV,Bilirubin_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,NaN


In [131]:
D_LABITEMS.query("ITEMID == 50883")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
83,210,210,84,50883,"Bilirubin, Direct",Blood,Chemistry,1968-7


In [68]:
D_LABITEMS[['bilirubin, total' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
38,165,165,39,50838,"Bilirubin, Total, Ascites",Ascites,Chemistry,14422-0
85,212,212,86,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
211,338,338,212,51012,"Bilirubin, Total, CSF",Cerebrospinal Fluid (CSF),Chemistry,1973-7
227,354,354,228,51028,"Bilirubin, Total, Body Fluid",Other Body Fluid,Chemistry,1974-5
248,375,375,249,51049,"Bilirubin, Total, Pleural",Pleural,Chemistry,14421-2


## Add White Blood Cell Count to dataset  

In [76]:
# 861, 1542, 220546, 51301, 51300, 1127

In [77]:
D_ITEMS.query("ITEMID == 861 | ITEMID == 1542 | ITEMID == 220546 | ITEMID == 1127")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
748,772,772,1149,861,"WBC (4-11,000)",NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
871,120,120,1272,1127,"WBC (4-11,000)",NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
1131,2810,2810,2994,1542,WBC,NaN,carevue,chartevents,Hematology,NaN,NaN,NaN
9538,11472,11472,12686,220546,WBC,WBC,metavision,chartevents,Labs,None,Numeric,NaN


In [78]:
D_ITEMS[['white' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2647,2737,2737,2921,3834,WhiteBloodC 4.0-11.0,NaN,carevue,chartevents,Heme/Coag,NaN,NaN,NaN


In [79]:
D_LABITEMS.query("ITEMID == 51300")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
499,626,626,500,51300,WBC Count,Blood,Hematology,26464-8


In [80]:
D_LABITEMS[['white' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
500,627,627,501,51301,White Blood Cells,Blood,Hematology,804-5


## Add Diastolic Blood Pressure to dataset  

In [81]:
# 224643, 225310, 220180, 8555, 220051, 8368, 8441, 8440

In [82]:
D_ITEMS.query("ITEMID == 225310 | ITEMID == 8555 | ITEMID == 8368 | ITEMID == 8441 | ITEMID == 8440")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
4829,4637,4637,4831,8368,Arterial BP [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4901,4709,4709,4903,8440,Manual BP [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4902,4710,4710,4904,8441,NBP [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
5016,4823,4823,5017,8555,Arterial BP #2 [Diastolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10598,12444,12444,13688,225310,ART BP Diastolic,ART BP Diastolic,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


In [83]:
D_ITEMS[['blood pressure' in str(s).lower() and 'diastolic' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9501,11503,11503,12717,220051,Arterial Blood Pressure diastolic,ABPd,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9519,11521,11521,12735,220180,Non Invasive Blood Pressure diastolic,NBPd,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
10150,11323,11323,13424,224643,Manual Blood Pressure Diastolic Left,Manual BPd L,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
11545,9441,9441,14618,227242,Manual Blood Pressure Diastolic Right,Manual BPd R,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


## Add Systolic Blood Pressure to dataset  

In [84]:
# 442, 227243, 224167, 220179, 225309

In [85]:
D_ITEMS.query("ITEMID == 442 | ITEMID == 225309")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
381,671,671,408,442,Manual BP [Systolic],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10597,12443,12443,13687,225309,ART BP Systolic,ART BP Systolic,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


In [86]:
D_ITEMS[['blood pressure' in str(s).lower() and 'systolic' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9500,11502,11502,12716,220050,Arterial Blood Pressure systolic,ABPs,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9518,11520,11520,12734,220179,Non Invasive Blood Pressure systolic,NBPs,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9958,9314,9314,13050,224167,Manual Blood Pressure Systolic Left,Manual BPs L,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
11546,9442,9442,14619,227243,Manual Blood Pressure Systolic Right,Manual BPs R,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


## Add Mean Blood Pressure to dataset  

In [88]:
# 456, 220181, 224, 225312, 220052, 52, 6702, 224322

In [89]:
D_ITEMS.query("ITEMID == 456 | ITEMID == 224 | ITEMID == 225312 | ITEMID == 52 | ITEMID == 6702 | ITEMID == 224322")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
31,321,321,58,52,Arterial BP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
195,485,485,222,224,IABP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
393,683,683,420,456,NBP Mean,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4124,5063,5063,4326,6702,Arterial BP Mean #2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10018,10164,10164,13110,224322,IABP Mean,MAP - Assisted,metavision,chartevents,IABP,mmHg,Numeric,NaN
10599,12445,12445,13689,225312,ART BP mean,ART BP mean,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


In [90]:
D_ITEMS[['blood pressure mean' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9502,11504,11504,12718,220052,Arterial Blood Pressure mean,ABPm,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN
9520,11522,11522,12736,220181,Non Invasive Blood Pressure mean,NBPm,metavision,chartevents,Routine Vital Signs,mmHg,Numeric,NaN


## Add PaCO2 (Partial pressure of carbon dioxide) to dataset  

In [91]:
# 226062, 778, 220235, 50818

In [115]:
D_ITEMS.query("ITEMID == 226062 | ITEMID == 778 | ITEMID == 220235")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
671,1517,1517,698,778,Arterial PaCO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN
9527,11529,11529,12743,220235,Arterial CO2 Pressure,PCO2 (Arterial),metavision,chartevents,Labs,mmHg,Numeric,NaN
11136,11644,11644,14344,226062,Venous CO2 Pressure,PCO2 (Venous),metavision,chartevents,Labs,mmHg,Numeric,NaN


In [93]:
D_LABITEMS.query("ITEMID == 50818")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
18,145,145,19,50818,pCO2,Blood,Blood Gas,11557-6


## Add PaO2 (partial pressure of oxygen) to dataset  

In [94]:
# 779

In [95]:
D_ITEMS[['pao2' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
423,713,713,450,490,PAO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
672,1518,1518,699,779,Arterial PaO2,NaN,carevue,chartevents,ABG,NaN,NaN,NaN


## Add FiO2 (fractional inspired oxygen) to dataset  

In [96]:
# 189, 727, 223835

In [97]:
D_ITEMS.query("ITEMID == 223835")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9767,12413,12413,12804,223835,Inspired O2 Fraction,FiO2,metavision,chartevents,Respiratory,None,Numeric,NaN


In [98]:
D_ITEMS[['fio2' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
159,449,449,186,185,FIO2 Alarm-High,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
160,450,450,187,186,FIO2 Alarm-Low,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
163,453,453,190,189,FiO2 (Analyzed),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
164,454,454,191,190,FiO2 Set,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
165,455,455,192,191,FiO2/O2 Delivered,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
649,1495,1495,676,727,Vision FiO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
796,45,45,1197,1040,BIpap FIO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
897,146,146,1298,1206,HFO FIO2:,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1377,2006,2006,3240,1863,HFO-FiO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1749,3243,3243,3612,2518,HFO- FIO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


## Add PaO/FiO2 ratio to dataset  

## Add Respiratory Rate to dataset   

In [99]:
# 224422, 618, 220210, 224689, 614, 651, 224690, 615

In [100]:
D_ITEMS.query("ITEMID == 224422 | ITEMID == 614 | ITEMID == 651 | ITEMID == 615")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
544,259,259,571,614,Resp Rate (Spont),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
545,260,260,572,615,Resp Rate (Total),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
577,1423,1423,604,651,Spon RR (Mech.),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10065,10413,10413,13339,224422,Spont RR,Spont RR,metavision,chartevents,Respiratory,bpm,Numeric,NaN


In [101]:
D_ITEMS[['respiratory rate' in str(s).lower()in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
548,263,263,575,618,Respiratory Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
549,1395,1395,576,619,Respiratory Rate Set,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9522,11524,11524,12738,220210,Respiratory Rate,RR,metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10181,10052,10052,13244,224688,Respiratory Rate (Set),Respiratory Rate (Set),metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10182,10053,10053,13245,224689,Respiratory Rate (spontaneous),Respiratory Rate (spontaneous),metavision,chartevents,Respiratory,insp/min,Numeric,NaN
10183,10054,10054,13246,224690,Respiratory Rate (Total),Respiratory Rate (Total),metavision,chartevents,Respiratory,insp/min,Numeric,NaN


## Add Temperature (Celsius) to dataset     

In [102]:
# 223761, 677, 676, 679, 678, 223762

In [103]:
D_ITEMS.query("ITEMID == 223761 | ITEMID == 678")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
602,1448,1448,629,678,Temperature F,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9720,12366,12366,12757,223761,Temperature Fahrenheit,Temperature F,metavision,chartevents,Routine Vital Signs,?F,Numeric,NaN


In [104]:
D_ITEMS[['temperature' in str(s).lower() and 'c' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
600,1446,1446,627,676,Temperature C,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
601,1447,1447,628,677,Temperature C (calc),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
603,1449,1449,630,679,Temperature F (calc),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4998,4813,4813,5007,8537,"Temp/Iso/Warmer [Temperature, degrees C]",NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9721,12367,12367,12758,223762,Temperature Celsius,Temperature C,metavision,chartevents,Routine Vital Signs,?C,Numeric,NaN
10172,10043,10043,13235,224674,Changes in Temperature,Changes in Temperature,metavision,chartevents,Toxicology,NaN,Text,NaN
11211,12254,12254,14446,226329,Blood Temperature CCO (C),Blood Temp CCO (C),metavision,chartevents,Routine Vital Signs,?C,Numeric,NaN
11492,9415,9415,14731,227054,TemperatureF_ApacheIV,TemperatureF_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),?F,Numeric,NaN


## Add Weight (kg) to dataset     

In [105]:
# 226531, 763, 224639, 226512

In [106]:
D_ITEMS.query("ITEMID == 226531 | ITEMID == 763 | ITEMID == 224639")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
661,1507,1507,688,763,Daily Weight,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
10146,11319,11319,13420,224639,Daily Weight,Daily Weight,metavision,chartevents,General,kg,Numeric,NaN
11259,11885,11885,14401,226531,Admission Weight (lbs.),Admission Weight (lbs.),metavision,chartevents,General,None,Numeric,NaN


In [107]:
D_ITEMS[['weight' in str(s).lower() and 'kg' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
2491,3842,3842,2765,3580,Present Weight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2494,3845,3845,2768,3583,Previous Weight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2575,3926,3926,2849,3693,Weight Kg,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2578,3929,3929,2852,3723,Birth Weight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2658,860,860,2214,4183,Birthweight (kg),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
11249,11875,11875,14391,226512,Admission Weight (Kg),Admission Weight (Kg),metavision,chartevents,General,kg,Numeric,NaN


## Add Heart Rate to dataset     

In [108]:
# 211, 220045

In [109]:
D_ITEMS[['heart rate' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
185,475,475,212,211,Heart Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2428,1897,1897,1078,3494,Lowest Heart Rate,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
9496,11498,11498,12712,220045,Heart Rate,HR,metavision,chartevents,Routine Vital Signs,bpm,Numeric,NaN
9497,11499,11499,12713,220046,Heart rate Alarm - High,HR Alarm - High,metavision,chartevents,Alarms,bpm,Numeric,NaN
9498,11500,11500,12714,220047,Heart Rate Alarm - Low,HR Alarm - Low,metavision,chartevents,Alarms,bpm,Numeric,NaN


## Add SpO2 (Oxygen Saturation) to dataset     

In [110]:
# 646, 50817, 834, 220277

In [111]:
D_ITEMS.query("ITEMID == 646 | ITEMID == 834 | ITEMID == 220277")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
572,1418,1418,599,646,SpO2,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
725,749,749,1126,834,SaO2,NaN,carevue,chartevents,ABG's,NaN,NaN,NaN
9530,12355,12355,12746,220277,O2 saturation pulseoxymetry,SpO2,metavision,chartevents,Respiratory,%,Numeric,NaN


In [112]:
D_LABITEMS.query("ITEMID == 50817")

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
17,144,144,18,50817,Oxygen Saturation,Blood,Blood Gas,20564-1


## Add Fluid Output - 4 hourly period to dataset      

## Add Mechanical Ventilation to dataset      

In [113]:
D_ITEMS[['ventilation' in str(s).lower() for s in D_ITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
10592,12438,12438,13682,225303,Mask Ventilation (Intubation),Mask Ventilation (Intubation),metavision,chartevents,Intubation,NaN,Text,NaN
10932,10213,10213,13977,225792,Invasive Ventilation,Invasive Ventilation,metavision,procedureevents_mv,2-Ventilation,None,Process,NaN
10933,10214,10214,13978,225794,Non-invasive Ventilation,Non-invasive Ventilation,metavision,procedureevents_mv,2-Ventilation,None,Process,NaN


In [114]:
D_LABITEMS[['ventilation' in str(s).lower() for s in D_LABITEMS['LABEL']]]

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
27,154,154,28,50827,Ventilation Rate,Blood,Blood Gas,NaN
